In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

LR，随机森林 已经尝试\
既然boosting 的RFC 超出了内存限制 \
n_estimator=800是我测试的上限树数量\
何不试试更好的gdbt算法实现呢？\
这个 notebook 尝试使用 XGB 并调参\
如果可能将在另一版本测试 lightgbm

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

X_train = train.iloc[:,1:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,1:]
X_train.shape, y_train.shape, X_test.shape


In [ ]:
y_train_int = np.array([int(item[-1])-1 for item in y_train])
y_train_int, y_train_int.shape

In [ ]:
def loss(y_true,y_pred):
    # gt is short for ground -truth
    # it is a matrix in which each line has only one 1 and others are 0
    # you can get it from get_dummies(df.target)
    # 
    # pred is from clf.pred_proba()
    
    assert y_pred.shape == y_true.shape
    
    sum = 0
    for i in y_pred * y_true:
        sum += np.log(
            min( max(10e-15, np.sum(i)), 1 - 10e-15)
        )
    return sum * (-1) * (1/y_pred.shape[0])

In [ ]:
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
xgb.__version__

In [ ]:
def my_scorer(y_true, y_pred_proba):
    y_true_mat = pd.get_dummies(y_true).values
    error = loss(y_true_mat, y_pred_proba)
    return error
the_scorer = make_scorer(my_scorer, needs_proba=True, greater_is_better=False)


In [ ]:
# try-1 
# regressor with scorer : needs_proba=False
# regressor only implemented predict method
# it can't output score matrix
# 
# try-2
# classifier with scorer: needs_proba=True
# it has method: predict_proba
# use it

# y_train_int needs to be encoded as 0~8

cv_params = {'learning_rate': [0.018, 0.02, 0.022], 'gamma':[0, 0.05, 0.1]}
other_params = {'learning_rate': 0.3, 'n_estimators': 800, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBClassifier(use_label_encoder=False, **other_params, eval_metric=['mlogloss'])
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring=the_scorer, verbose=3, cv=3)
optimized_GBM.fit(X_train, y_train_int)

In [ ]:
for result in optimized_GBM.cv_results_:
    print(result, optimized_GBM.cv_results_[result])
optimized_GBM.best_params_

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit

In [ ]:
y_pred = optimized_GBM.predict_proba(X_test)
y_pred

In [ ]:
my_submit = pd.DataFrame()
# my_submit = pd.DataFrame(y_pred,columns=submit.columns[1:])
my_submit['id'] = submit.id
my_submit[submit.columns[1:]] = y_pred
my_submit

In [ ]:
my_submit.to_csv('my_submit_xgb.csv',index=False)
reload = pd.read_csv('my_submit_xgb.csv')
reload
